### Данный Jupiter-ноутбук предназначен для запуска на Jupyter-сервере внутри AI Cloud. Если вы хотите выполнить те же самые действия из удаленного Jupyter-сервера, посмотрите пример отправки задач на обучение через Training Job API.

# 0. Подключение библиотеки для работы с кластером и сервисом

In [1]:
try:
    import client_lib
except ImportError:
    raise RuntimeError("Скрипт не предназначен для запуска вне кластера")

In [2]:
import requests

In [3]:
def save_file(url, filename):
    # Download file and place it on local storage
    r = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f"{filename} downloaded from {url}")

# 1. Размещаем модель и данные на локальные диски кластера 

По умолчанию рабочая директория пользователя установлена в папку `/home/jovyan`

In [4]:
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch/mnist.npz", "mnist.npz")

mnist.npz downloaded from https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch/mnist.npz


In [5]:
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch/requirements.txt", "requirements.txt")

requirements.txt downloaded from https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch/requirements.txt


In [6]:
save_file("https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/job_launch/tensorflow_mnist_estimator.py", "tensorflow_mnist_estimator.py")

tensorflow_mnist_estimator.py downloaded from https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/job_launch/tensorflow_mnist_estimator.py


# 2. Сборка кастомного образа с нужными библиотеками

##### Содержимое файла requirements.txt

In [8]:
%cat /home/jovyan/quick-start/job_launch/requirements.txt

# tensorflow-gpu 1.15 already installed in registry.aicloud.sbcp.ru/horovod-tf15
# with other libraries, we add one more:
requests

##### Запуск сборки кастомного образа с необходимыми библиотеками

In [9]:
job = client_lib.ImageBuildJob(
                    from_image='cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0', # базовый образ
                    requirements_file='/home/jovyan/quick-start/job_launch/requirements.txt' # файл с зависимостями для кастомного образа
)

In [10]:
job.submit()

'ImageBuildJob "{\'image\': \'cr.msk.sbercloud.ru/eec0971f-da40-49ba-a962-d08ec3a53a0a/job-custom-image-0fb9d1\', \'name\': \'image-build-job-m9ds2\', \'status\': \'ok\'}" created'

In [11]:
job.new_image # идентификатор кастомного образа

'cr.msk.sbercloud.ru/eec0971f-da40-49ba-a962-d08ec3a53a0a/job-custom-image-0fb9d1'

In [ ]:
job.logs() # просмотр логов сборки образа в интерактивном режиме

INFO[0005] Resolved base name cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0 to cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0 
INFO[0005] Resolved base name cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0 to cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0 
INFO[0005] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0 
INFO[0005] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0 
INFO[0005] Built cross stage deps: map[]                
INFO[0005] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0 
INFO[0006] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0 


# 3. Запуск задачи обучения на кластере

#### Создание задачи обучения и отправка ее на кластер

Параметр `base_image` предназначен для запуска задачи на кластере в своем кастомном образе (см. пункт 2 выше). Переменная `job.new_image` содержит название собранного кастомного образа (вида: `registry.aicloud.sbcp.ru/ai0000001-0090/09aa1302-6d95-4f04-8f23-71013679aa00`).

По окончании сборки кастомного образа его можно указать в поле `base_image` и отправить задачу обучения на кластер с использованием этого образа:
```python
mnist_tf_run = client_lib.Job(base_image=job.new_image, # кастомный образ
                              script='/home/jovyan/quick-start/job_launch/tensorflow_mnist_estimator.py',
                              n_workers=2, n_gpus=4, warm_cache=False)
```

In [4]:
# base_image='registry.aicloud.sbcp.ru/base/horovod-cuda10.0-tf1.15.0' -
#     базовый образ, можно заменить на свой (смотри выше)
mnist_tf_run = client_lib.Job(base_image='cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0',
                              script='/home/jovyan/quick-start/job_launch/tensorflow_mnist_estimator.py',
                              n_workers=2, n_gpus=4, warm_cache=False)

#### В предыдущей строке мы указали образ, в рамках которого будет исполняться задача (base_image), скрипт, который будет запущен (script), а также количество рабочих узлов кластера (n_workers) и количество GPU-карт на каждом (n_gpus). 
#### В этом случае под задачу будет запущено 2 рабочих узла по 4 GPU карты (ИТОГО - 8 GPU).
#### Теперь вызовем на объекте метод .submit() и отправим задачу в очередь исполнения на кластере

In [5]:
mnist_tf_run.submit()

'Job "lm-mpi-job-7912945d-4cdc-45ef-b6e7-3b5fe683029a" created'

#### Вы можете просмотреть список ваших задач и их статус с помощью следующей команды:

In [13]:
client_lib.jobs()

2020-02-28T14:54:31Z : lm-mpi-job-0170cd13-0dde-4741-9b02-ce9f0eee1349 : Completed
2020-02-20T08:39:16Z : lm-mpi-job-0ac8ff95-7c6d-4c20-bf4f-19cdec0070ca : Completed
2020-03-05T08:17:51Z : lm-mpi-job-0dbfcf60-ebf4-4020-bc82-d93862d597b4 : Completed
2020-03-03T13:28:02Z : lm-mpi-job-0e875e8d-01b1-4ed9-a4a4-6d4de951fb00 : Completed
2020-03-03T15:46:37Z : lm-mpi-job-1702cf84-60f1-4ca9-bba6-a22605c09558 : Completed
2020-03-03T13:33:09Z : lm-mpi-job-27d99673-f827-441a-994c-7cfb4e611679 : Completed
2020-02-13T16:39:04Z : lm-mpi-job-335a09be-1ecb-4661-920b-857d21e6f850 : Completed
2020-02-04T09:58:29Z : lm-mpi-job-4ef73dd8-9d3d-487b-9570-7d66df018ef6 : Completed
2020-02-04T09:40:27Z : lm-mpi-job-53783174-5696-4f40-a5ff-bba8dd5c3963 : Completed
2020-02-27T15:42:14Z : lm-mpi-job-5801b303-f94f-41b8-a928-dca1cc55a068 : Completed
2020-02-27T10:38:49Z : lm-mpi-job-5f7e27d6-8391-4ccc-9bb8-c39a5b7928fe : Completed
2020-02-13T14:25:28Z : lm-mpi-job-73cdd281-f0f1-48a1-83cc-74930836cf54 : Completed
2020

#### Логи задачи

In [7]:
mnist_tf_run.logs()

Job in queue. Try later


##### или по названию задачи

In [8]:
client_lib.logs("lm-mpi-job-7912945d-4cdc-45ef-b6e7-3b5fe683029a")

Job in queue. Try later


#### (ЕСЛИ НЕОБХОДИМО ПРЕРВАТЬ ЗАДАЧУ) Варианты остановки задачи

In [10]:
mnist_tf_run.kill()

'Job "lm-mpi-job-a0f7fdc7-3b27-45aa-a709-4f4821b887a1" deleted'

In [38]:
client_lib.kill('lm-mpi-job-7912945d-4cdc-45ef-b6e7-3b5fe683029a')

'Job "lm-mpi-job-3f332e07-ebca-4e22-afc8-852d6c53a922" deleted'

# 4. Сохранение промежуточных результатов обучения модели

Если в процессе обучения модели пользователь сохраняет промежуточные результаты (checkpoints) обучения, они попадают в папку `/home/jovyan/quick-start/job_launch/checkpoints_tf/mnist_convnet_model/`. Их можно скачать через веб-интерфейс Jupyter-ноутбука или скопировать из локально доступной файловой системы в хранилище S3. Внутри кода задачи обучения можно сохранять метрики модели с помощью mlflow (пример — в коде `tensorflow_mnist_estimator.py`).

## Выгрузка результатов обучения модели с NFS на S3

##### Указываем параметры доступа к бакету S3

Для перемещения данных из локального хранилища `/home/jovyan` в объектное хранилище S3 необходимо указать параметры доступа к бакету S3. Эти данные можно найти на портале portal.sbercloud.ru в параметрах заказанной услуги AI Cloud ("Мои услуги"->"AI Cloud").

В одинарных кавычках `''` введите  по порядку: `S3 namespace`, `S3 access key`, `S3 security key`

In [15]:
# Specify S3 credentials as follows
# save_aws_credentials('b175178b-f537-4a1c-8977-7871bb9dc448-namespace', 'b13f3480-a701-48fb-85c5-b4a3b36f3713', 'BPBdF5E9CGAFv+rmBakFc+R0krVCHd/u4rPsqfG8')
client_lib.save_aws_credentials('', '', '')

Ниже укажем бакет s3, в который будут копироваться файлы

In [ ]:
# Specify S3 bucket
# s3_bucket = "s3://9b8d0e96-6461-47cf-9507-21ede838c9ac-bucket/"
s3_bucket = ""

In [ ]:
# Перемещение отдельных файлов
# client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch/checkpoints_tf/mnist_convnet_model/model.ckpt-10937.data-00000-of-00001", s3_bucket).submit()
# client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch/checkpoints_tf/mnist_convnet_model/model.ckpt-10937.index", s3_bucket).submit()
# client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch/checkpoints_tf/mnist_convnet_model/model.ckpt-10937.meta", s3_bucket).submit()

In [19]:
# Перемещение папки с файлами
client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch/checkpoints_tf/mnist_convnet_model/", s3_bucket, recursive=True).submit()

'S3CopyJob "{\'name\': \'copying-job-wvgz2\', \'status\': \'ok\'}" created'

Если все было сделано правильно, то промежуточные результаты обучения модели сохранятся в объектном хранилище S3, из которого можно будет впоследствии восстановить объект с моделью